# Comment utiliser le dataloader

## Générer un train et test set

In [1]:
from tools.data import DreemDatasets

Les paramètres sont :

- chemin vers les données
- chemin vers les cibles
- `keep_datasets` Liste des datasets à garder, parmi les suivants :
    * `eeg_1` - EEG in frontal position sampled at 50 Hz -> 1500 values
    * `eeg_2` - EEG in frontal position sampled at 50 Hz -> 1500 values
    * `eeg_3` - EEG in frontal position sampled at 50 Hz -> 1500 values
    * `eeg_4` - EEG in frontal-occipital position sampled at 50 Hz -> 1500 values
    * `eeg_5` - EEG in frontal-occipital position sampled at 50 Hz -> 1500 values
    * `eeg_6` - EEG in frontal-occipital position sampled at 50 Hz -> 1500 values
    * `eeg_7` - EEG in frontal-occipital position sampled at 50 Hz -> 1500 values
    * `accelerometer_x` - Accelerometer along x axis sampled at 10 Hz -> 300 values
    * `accelerometer_y` - Accelerometer along y axis sampled at 10 Hz -> 300 values
    * `accelerometer_z` - Accelerometer along z axis sampled at 10 Hz -> 300 values
    * `pulse_oximeter_infrared` - Pulse oximeter infrared channel sampled at 10 Hz -> 300 values
- `split_train_val` Pourcentage pour partager le train set et validation set
- `seed` Une seed pour la reproductibilité
- `balance_data` si vrai, équilibre le dataset pour avoir le nombre de donnée par classe
- `size` Une taille maximale pour le dataset (si non renseignée, tout le dataset)
- `transforms` des transformations à appliquer aux données (voir la partie transformation)
- `transforms_val` si renseignée, `transforms` sera pour le train et `transforms_val` pour la validation. Sinon, même transformation que `transforms`.


### Avec context manager

In [15]:
with DreemDatasets('dataset/train.h5', 'dataset/train_y.csv', split_train_val=0.8, seed=0,
                   size=5000, keep_datasets=['eeg_1']) as (train_set, val_set):
    pass  # Faire des choses

### Sans context manager

In [16]:
train_set, val_set = DreemDatasets('dataset/train.h5', 'dataset/train_y.csv', 
                                   split_train_val=0.8, seed=0, keep_datasets=['eeg_1']).get()

train_set.load_data()  # Load les données en mémoire
# train_set.load_data(npy_file)  # Load les données depuis un fichier de sauvegarde

val_set.load_data()

# Ne pas oublier de fermer les datasets

train_set.close()  # Ne ferme que les fichiers h5. Si mis en mémoire, on a toujours accès aux données !
val_set.close()

Loading data in memory...
Done.
Loading data in memory...
Done.


### Transformations

C'est un dictionnaire avec une fonction prenant un batch de signal en paramètre et renvoyant le batch transformé en valeur, et le nom du dataset en clé.

Exemple :

In [12]:
transformations = {
    "eeg_1": lambda signals: signals[:, 0]
}

### Des transformations

In [13]:
# Extrait le spectre fenêtré
from preprocessing.signals import ExtractSpectrum
# Extrait les bandes de fréquences 
from preprocessing.signals import ExtractBands
# Extrait des features
from preprocessing.features import ExtractFeatures
# Compose des fonctions entre elles
from preprocessing import Compose

extract_spectrum = ExtractSpectrum(window=10, sampling_freq=50)
extract_bands = ExtractBands(bands='*')  # toutes les bandes
# Extract features peut directement extraire les bandes
extract_features = ExtractFeatures(bands='*', features=['mmd', 'esis'])

transformations = {
    "eeg_1": extract_spectrum,
    "eeg_2": extract_bands,
    "eeg_3": extract_features
}

### Récupérer des données

In [18]:
d50hz, d10hz, target = train_set[0]  # Une valeur
# Dimension nb_datasets x tailles_features

data50hz, data10hz, targets = train_set[0:10]  # 10 valeurs
# Dimension nb_datasets x nb_elements x tailles_features

## Importer le test

A corriger...

In [21]:
from tools.data import DreemDataset

test_set = DreemDataset('dataset/test.h5', keep_datasets=['eeg_1'])

test_set.load_data()

test_set.close()

Loading data in memory...


Exception ignored in: <function DreemDataset.__del__ at 0x7f8250c92400>
Traceback (most recent call last):
  File "/run/media/bdvllrs/Data/Documents/Supelec/OMA/MLC/Projet/tools/data.py", line 213, in __del__
    def __exit__(self, exc_type, exc_val, exc_tb):
  File "/run/media/bdvllrs/Data/Documents/Supelec/OMA/MLC/Projet/tools/data.py", line 204, in close
    def close(self):
AttributeError: 'DreemDataset' object has no attribute 'data'


ValueError: Not a dataset (not a dataset)